# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure ***MATERIALS_DIR*** points to the directory where you extracted the Zip file.
* Make sure all your paths are **relative to ** ***MATERIALS_DIR*** and **NOT hard-coded** in your code.

In [1]:
# imports
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
import os, os.path
import shutil

In [2]:
MATERIALS_DIR = r"C:\DSS_Fall2017_Assign2"

# Put other path constants here

DOCUMENTS_DIR = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\documents")
INDEX_DIR2 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\index1")
QUER_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\topics\gov.topics")
QRELS_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\qrels\gov.qrels")
OUTPUT_FILE2 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\myres")
TREC_EVAL = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\trec_eval\trec_eval.exe")
INDEX_DIR3 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\index2")
OUTPUT_FILE3 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\myres2")
INDEX_DIR4 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\index3")
OUTPUT_FILE4 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\myres3")

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 
(a)  "Pk" (Percision for the first k documents returned)

### Q1 (b): Provide answer to Q1 
(b) When users search specific queries, they don't go through all the results. They want to find the right answer after opening few pages. Therefore, percision for the first 5-10 documents returned should be maximized.

## Question 2

### Q2 (a): Write your code below

In [3]:
# first, define a Schema for the index

mySchema2 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

In [4]:
# if index exists - remove it
if os.path.isdir(INDEX_DIR2):
    shutil.rmtree(INDEX_DIR2)

# create the directory for the index
os.makedirs(INDEX_DIR2)

# create index
INDEX_Q2 = index.create_in(INDEX_DIR2, mySchema2)

In [5]:
# Is it empty?
print("Index is empty?", INDEX_Q2.is_empty())

# How many files indexed?
print("Number of indexed files:", INDEX_Q2.doc_count())

Index is empty? True
Number of indexed files: 0


In [6]:
# First, lets review the documents in our sample dataset
!dir $DOCUMENTS_DIR

 Volume in drive C has no label.
 Volume Serial Number is 10DF-697A

 Directory of C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents

09/29/2017  04:23 PM    <DIR>          .
09/29/2017  04:23 PM    <DIR>          ..
09/29/2017  04:21 PM            10,244 .DS_Store
09/29/2017  04:21 PM    <DIR>          00
09/29/2017  04:21 PM    <DIR>          01
09/29/2017  04:21 PM    <DIR>          02
09/29/2017  04:21 PM    <DIR>          03
09/29/2017  04:21 PM    <DIR>          04
09/29/2017  04:21 PM    <DIR>          05
09/29/2017  04:21 PM    <DIR>          06
09/29/2017  04:21 PM    <DIR>          07
09/29/2017  04:21 PM    <DIR>          08
09/29/2017  04:21 PM    <DIR>          09
09/29/2017  04:22 PM    <DIR>          10
09/29/2017  04:22 PM    <DIR>          11
09/29/2017  04:22 PM    <DIR>          12
09/29/2017  04:22 PM    <DIR>          13
09/29/2017  04:22 PM    <DIR>          14
09/29/2017  04:22 PM    <DIR>          15
09/29/2017  04:22 PM    <DIR>          16
09/2

In [7]:
# first we build a list of all the full paths of the files in DOCUMENTS_DIR
filesToIndex = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex.extend(filePaths)

In [8]:
# print the first 5 paths to make sure it worked
print("\n".join(filesToIndex[:5]))

C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\00\G00-00-0088569
C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\00\G00-00-0114013
C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\00\G00-00-0124389
C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\00\G00-00-0158061
C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\00\G00-00-0165832


In [9]:
# count files to index
print("number of files:", len(filesToIndex))

number of files: 4078


In [10]:
# open writer
myWriter2= writing.BufferedWriter(INDEX_Q2, period=20, limit=1000)

try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex):
        with open(filePath, "r", encoding="utf-8") as f:
            fileContent = f.read()
            myWriter2.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")
  
finally:
    # save the index
    myWriter2.close()

already indexed: 1
already indexed: 1001
already indexed: 2001
already indexed: 3001
already indexed: 4001
done indexing.


In [11]:
# print the topic file
!cat $QUER_FILE

1 mining gold silver coal
2 juvenile delinquency
4 wireless communications
6 physical therapists
7 cotton industry
9 genealogy searches
10 Physical Fitness
14 Agricultural biotechnology
16 Emergency and disaster preparedness assistance
18 Shipwrecks
19 Cybercrime, internet fraud, and cyber fraud
22 Veteran's Benefits
24 Air Bag Safety
26 Nuclear power plants
28 Early Childhood Education


In [12]:
# define a query parser for the field "file_content" in the index
QP_Q2 = QueryParser("file_content", schema=INDEX_Q2.schema)
SEARCHER_Q2 = INDEX_Q2.searcher()

In [13]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()

# create an output file to which we'll write our results
outputTRECFile2 = open(OUTPUT_FILE2, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1)) 
    topicQuery = QP_Q2.parse(topic_phrase)
    topicResults = SEARCHER_Q2.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile2.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile2.close()
topicsFile.close()

In [14]:
!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE2

num_ret               	1	1
num_rel               	1	5
num_rel_ret           	1	0
map                   	1	0.0000
Rprec                 	1	0.0000
bpref                 	1	0.0000
recip_rank            	1	0.0000
iprec_at_recall_0.00  	1	0.0000
iprec_at_recall_0.10  	1	0.0000
iprec_at_recall_0.20  	1	0.0000
iprec_at_recall_0.30  	1	0.0000
iprec_at_recall_0.40  	1	0.0000
iprec_at_recall_0.50  	1	0.0000
iprec_at_recall_0.60  	1	0.0000
iprec_at_recall_0.70  	1	0.0000
iprec_at_recall_0.80  	1	0.0000
iprec_at_recall_0.90  	1	0.0000
iprec_at_recall_1.00  	1	0.0000
P_5                   	1	0.0000
P_10                  	1	0.0000
P_15                  	1	0.0000
P_20                  	1	0.0000
P_30                  	1	0.0000
P_100                 	1	0.0000
P_200                 	1	0.0000
P_500                 	1	0.0000
P_1000                	1	0.0000
num_ret               	10	16
num_rel               	10	1
num_rel_ret           	10	1
map                   	10	0.1667
Rprec                 	10	0.0000


In [15]:
#!$TREC_EVAL -h $QRELS_FILE $OUTPUT_FILE2

In [16]:
!head -n 150 $OUTPUT_FILE2

1 Q0 G00-90-0342721 0 26.645398 test
2 Q0 G00-22-3396139 0 17.262139 test
2 Q0 G00-76-0415824 1 10.597055 test
2 Q0 G00-78-1531079 2 8.778648 test
2 Q0 G00-15-1718631 3 8.076860 test
2 Q0 G00-70-2787853 4 6.788751 test
2 Q0 G00-74-1394517 5 3.368380 test
4 Q0 G00-99-2247765 0 16.449155 test
4 Q0 G00-85-1525415 1 13.364613 test
4 Q0 G00-05-1218739 2 12.956314 test
4 Q0 G00-09-0774298 3 11.781349 test
4 Q0 G00-56-4151981 4 11.367248 test
4 Q0 G00-21-2229498 5 10.743958 test
4 Q0 G00-98-4068688 6 10.464865 test
4 Q0 G00-47-2117970 7 10.213356 test
4 Q0 G00-67-0152545 8 8.392871 test
4 Q0 G00-06-1757034 9 6.431556 test
4 Q0 G00-78-2551063 10 3.955775 test
4 Q0 G00-84-0274223 11 2.068438 test
6 Q0 G00-26-3134051 0 13.996502 test
6 Q0 G00-59-0786269 1 13.853934 test
6 Q0 G00-60-3914816 2 11.345260 test
6 Q0 G00-21-0649032 3 5.955903 test
6 Q0 G00-45-4032177 4 5.937137 test
7 Q0 G00-70-0954490 0 16.517930 test
7 Q0 G00-59-2927976 1 16.234779 test
7 Q0 G00-75-1015577 2 11.178929 test
7 Q0 G00-

In [17]:
# print the first 10 lines in the qrels file
!head -n 3000 $QRELS_FILE

1 0 G00-00-0681214 0
1 0 G00-00-0945765 0
1 0 G00-00-1006224 1
1 0 G00-00-1591495 0
1 0 G00-00-2764912 0
1 0 G00-00-3253540 0
1 0 G00-00-3717374 0
1 0 G00-01-0270065 0
1 0 G00-01-0400712 0
1 0 G00-01-0682299 0
1 0 G00-01-2154945 0
1 0 G00-01-2689026 0
1 0 G00-01-2898660 0
1 0 G00-02-0146077 0
1 0 G00-02-0351712 0
1 0 G00-02-0510219 0
1 0 G00-02-0555602 0
1 0 G00-02-0901987 1
1 0 G00-02-1239993 0
1 0 G00-02-3981961 0
1 0 G00-02-4057099 0
1 0 G00-03-0366425 0
1 0 G00-03-0697220 0
1 0 G00-03-0931579 0
1 0 G00-03-1585062 0
1 0 G00-03-1898526 1
1 0 G00-04-0296440 0
1 0 G00-04-0767639 0
1 0 G00-04-0971232 0
1 0 G00-04-1046533 0
1 0 G00-04-1562575 0
1 0 G00-04-1864045 0
1 0 G00-04-3558844 0
1 0 G00-04-4166204 0
1 0 G00-05-0619345 0
1 0 G00-05-0623935 0
1 0 G00-05-1315394 0
1 0 G00-05-2231767 0
1 0 G00-05-2357004 0
1 0 G00-05-2948873 0
1 0 G00-05-2988698 0
1 0 G00-05-3051665 0
1 0 G00-05-3539237 0
1 0 G00-06-0872838 0
1 0 G00-06-2562505 0
1 0 G00-06-3965004 0
1 0 G00-06-4038523 0
1 0 G00-07-11

7 0 G00-06-1913581 0
7 0 G00-06-2562505 0
7 0 G00-06-2661322 0
7 0 G00-06-2965032 0
7 0 G00-06-3054024 0
7 0 G00-06-3652121 0
7 0 G00-07-0261374 0
7 0 G00-07-0393523 0
7 0 G00-07-1561076 0
7 0 G00-07-1993864 0
7 0 G00-07-2561555 0
7 0 G00-07-2654723 0
7 0 G00-07-2687847 0
7 0 G00-07-3318478 0
7 0 G00-07-3659195 0
7 0 G00-07-3853050 0
7 0 G00-07-3972872 0
7 0 G00-07-4009621 1
7 0 G00-07-4080218 0
7 0 G00-08-0085414 0
7 0 G00-08-0317811 0
7 0 G00-08-1048959 0
7 0 G00-08-1084598 0
7 0 G00-08-1594396 0
7 0 G00-08-1609604 0
7 0 G00-08-1667883 0
7 0 G00-08-2258958 0
7 0 G00-08-2472523 0
7 0 G00-08-2666680 0
7 0 G00-08-3234501 0
7 0 G00-08-3809731 0
7 0 G00-08-3960707 0
7 0 G00-09-0245064 0
7 0 G00-09-1193469 0
7 0 G00-09-1560577 0
7 0 G00-09-3195749 0
7 0 G00-09-3199095 0
7 0 G00-09-3375933 0
7 0 G00-10-0281044 0
7 0 G00-10-0814583 0
7 0 G00-10-1055581 0
7 0 G00-10-2836439 0
7 0 G00-10-2917540 0
7 0 G00-10-3137809 0
7 0 G00-10-3149786 0
7 0 G00-10-3222851 0
7 0 G00-10-3302265 1
7 0 G00-10-38

26 0 G00-11-2000231 0
26 0 G00-11-2005001 0
26 0 G00-11-2958190 0
26 0 G00-12-0399012 0
26 0 G00-12-1831266 0
26 0 G00-13-1543158 0
26 0 G00-14-1375985 0
26 0 G00-14-2310358 0
26 0 G00-15-0501460 0
26 0 G00-16-3952320 0
26 0 G00-17-3027297 0
26 0 G00-18-2332031 0
26 0 G00-20-1122738 0
26 0 G00-22-0206671 0
26 0 G00-26-0622450 0
26 0 G00-28-2250792 0
26 0 G00-30-0129773 0
26 0 G00-30-1518511 0
26 0 G00-30-1962408 0
26 0 G00-31-0699333 0
26 0 G00-32-0376398 0
26 0 G00-33-2469103 0
26 0 G00-35-0922387 0
26 0 G00-35-2190090 0
26 0 G00-35-3237468 0
26 0 G00-37-2508472 0
26 0 G00-39-0996057 0
26 0 G00-43-3227387 0
26 0 G00-44-2379326 0
26 0 G00-45-0332043 0
26 0 G00-49-0195605 0
26 0 G00-49-0846427 0
26 0 G00-49-1312433 0
26 0 G00-50-1075346 0
26 0 G00-50-2186799 0
26 0 G00-52-2991521 0
26 0 G00-53-0187653 0
26 0 G00-53-1684082 0
26 0 G00-54-3304332 0
26 0 G00-55-3690786 0
26 0 G00-55-3900830 0
26 0 G00-56-0327569 0
26 0 G00-56-0479901 0
26 0 G00-58-2466520 0
26 0 G00-59-3730691 0
26 0 G00-6

In [18]:
# let count the same words again
myReader1 = INDEX_Q2.reader()
print("# docs with 'juvenile'", myReader1.doc_frequency("file_content", "juvenile"))

# docs with 'juvenile' 19


In [19]:
# False positive :G00-34-3591274 0 34.092076 test - topic 16
#topic_sixteen= r"C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\34\G00-34-3591274"

#topic_sixteen= open (topic_sixteen, "r").read().splitlines()
#print (topic_sixteen)

In [20]:
print (topics)

['1 mining gold silver coal', '2 juvenile delinquency', '4 wireless communications', '6 physical therapists', '7 cotton industry', '9 genealogy searches', '10 Physical Fitness', '14 Agricultural biotechnology', '16 Emergency and disaster preparedness assistance', '18 Shipwrecks', '19 Cybercrime, internet fraud, and cyber fraud', "22 Veteran's Benefits", '24 Air Bag Safety', '26 Nuclear power plants', '28 Early Childhood Education']


### Q2 (b): Provide answer to Q2 

(b) P5 for all documents 0.0714

### Q2 (c): Provide answer to Q2
(c) 

Topics did very well: '18 Shipwrecks'

Topics did very badly: '1 mining gold silver coal', '16 Emergency and disaster preparedness assistance',
'2 juvenile delinquency', '28 Early Childhood Education', '6 physical therapists', '7 cotton industry', '9 genealogy searches'

## Question 3

### Q3 (a): Provide answer to Q3 
(a) Problems with previous document ranking:

Capital Letter: Some queries or some words in documents are in capital letter. Similar words with different capitalization do not match (Topic 28 as an example)

Stopwords: Topic 19 contains "and" word. Documents that have more count of "and", they can be ranked higher; therefore, stopwords should be removed.
 
Word's root: Words in plural forms or words with similar roots should be counted as relevant.




False positive cases contain the exact word in the query, so they retrieved. However, they are not relevent to the query.
Falsenegative cases may not cotain the exact form of the words in the query; however, they contain synonyms or roots of the topic.


I have chosen topic 28 (Early Childhood Education) to find false positive and False negative cases. 

G00-75-2371200 : This is False positive case, which ranked highest for the 28th query, while this document was irrelevant. This query only matched with documents that have similar capitalizations.Therefore, only limited and non relevent documents retrieved for this query.

G00-54-2576117: This is False Negative case, which did not retrieved for the 28th query, while this document was relevant. For this document, the document contains words such as child, children, childhood, child care, child-care and etc. However, only documents retrieved that contain the exact word of "Childhood". 

In [21]:
QP_Q2.parse("Early Childhood Education")

And([Term('file_content', 'Early'), Term('file_content', 'Childhood'), Term('file_content', 'Education')])

In [22]:
#False positive for topic 28
topic_o= r"C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\75\G00-75-2371200"

topic_o= open (topic_o, "r").read().splitlines()
print (topic_o)

['http://www.cdpac.ca.gov/files/Resources.html', '', '   [CDPAC_Resources_01.gif]', '', '', '   [CDPAC_Resources_02.gif] CDPAC Home Page', '   [CDPAC_Resources_06.gif] Who We Are', '   [CDPAC_Resources_08.gif] Local Planning', '   [CDPAC_Resources_10.gif] Monthly Meetings', '   [CDPAC_Resources_14.gif] Legislation', '   [CDPAC_Resources_16.gif] Publications', '   [CDPAC_Resources_18.gif] Resources', '   [CDPAC_Resources_20.gif] E-Mail Us', '   [CDPAC_Resources_22.gif]', '', '', '   Job Announcements & Duty Statements', '   *** There are currently no Job Announcements ***', '   Resource Links', '', '   blue_dot.gif (336 bytes) American Academy of Pediatrics', '   blue_dot.gif (336 bytes) American Medical Association', '   blue_dot.gif (336 bytes) American Public Human Services Association', '   blue_dot.gif (336 bytes) AT&T Foundation', '   blue_dot.gif (336 bytes) Brain Awareness Information', '   blue_dot.gif (336 bytes) Bureau of Labor Statistics', '   blue_dot.gif (336 bytes) Califo

In [23]:
#false Negative example for topic 28

topic_oe= r"C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\54\G00-54-2576117"

topic_oe= open (topic_oe, "r").read().splitlines()
print (topic_oe)

['http://goldmine.cde.ca.gov/cyfsbranch/chssco/', '', '                        California Dept. of Education', '          Work Plan   | Advisory Committee  |  BRIDGES  | CPR | Search', '', '           CHSSCO', '', '', '                 California Head Start State Collaboration Office', '', '                 Michael Silver, Project Director', '                 Michael Zito, Project Coordinator', '', '', '   Hot topics', '', '             CHSA Annual Conference & Leadership Day', '', '             Ninth Annual Family Literacy Conference', '', '', '   An Overview of Head Start', '', '   Head Start is a national program that provides comprehensive', '   developmental services for low-income children from birth to entry in', '   elementary school. Over a span of more than thirty years, Head Start', '   has provided educational, social, medical, dental, nutrition, mental', '   health services, and parent involvement activities to almost 18 million', '   children nationwide.', '', '   Head St

### Q3 (b): Write your code below

In [24]:
import nltk
from nltk.stem import *

In [25]:
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to C:\Windows\ServiceProfiles\
[nltk_data]     LocalService\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3

# we also probably want to break phrases like "whoosh.analysis" into "whoosh" and "analysis"
# so we add IntraWordFilter
#stmLwrStpIntraAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | StemFilter()

#mySchema3 = Schema(file_path = ID(stored=True),
#                   file_content = TEXT(analyzer = stmLwrStpIntraAnalyzer))

#stmLwrStpIntraAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | StemFilter()
#[token.text for token in stmLwrStpIntraAnalyzer("juvenile minerals mine")]

In [27]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [28]:
# For topic two, minerals and mine should have the same root

In [29]:
Filter3 = RegexTokenizer() | CustomFilter(LancasterStemmer().stem)
[token.text for token in Filter3("We are mine Mining going to do Text Analysis with whoosh.analysis")]



mySchema3 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = Filter3))


In [30]:
# if index exists - remove it
if os.path.isdir(INDEX_DIR3):
    shutil.rmtree(INDEX_DIR3)

# create the directory for the index
os.makedirs(INDEX_DIR3)

# create index or open it if already exists
INDEX_Q3  = index.create_in(INDEX_DIR3, mySchema3)

In [31]:
# first we build a list of all the full paths of the files in DOCUMENTS_DIR
filesToIndex = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex.extend(filePaths)

In [32]:
# open writer
myWriter3 = writing.BufferedWriter(INDEX_Q3 , period=None , limit=1000)

try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex):
        with open(filePath, "r", encoding="utf-8") as f:
            fileContent = f.read()
            myWriter3.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter3.close()

already indexed: 1
already indexed: 1001
already indexed: 2001
already indexed: 3001
already indexed: 4001
done indexing.


In [33]:
# define a query parser for the field "file_content" in the index
QP_Q3 = QueryParser("file_content", schema=INDEX_Q3.schema)
SEARCHER_Q3 = INDEX_Q3.searcher()

In [34]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()

# create an output file to which we'll write our results
outputTRECFile3 = open(OUTPUT_FILE3, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
    topicQuery = QP_Q3.parse(topic_phrase)
    topicResults = SEARCHER_Q3.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile3.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile3.close()
topicsFile.close()

In [35]:
QP_Q3.parse("Early Childhood Education")

And([Term('file_content', 'ear'), Term('file_content', 'child'), Term('file_content', 'educ')])

In [36]:
!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE3

num_ret               	1	3
num_rel               	1	5
num_rel_ret           	1	0
map                   	1	0.0000
Rprec                 	1	0.0000
bpref                 	1	0.0000
recip_rank            	1	0.0000
iprec_at_recall_0.00  	1	0.0000
iprec_at_recall_0.10  	1	0.0000
iprec_at_recall_0.20  	1	0.0000
iprec_at_recall_0.30  	1	0.0000
iprec_at_recall_0.40  	1	0.0000
iprec_at_recall_0.50  	1	0.0000
iprec_at_recall_0.60  	1	0.0000
iprec_at_recall_0.70  	1	0.0000
iprec_at_recall_0.80  	1	0.0000
iprec_at_recall_0.90  	1	0.0000
iprec_at_recall_1.00  	1	0.0000
P_5                   	1	0.0000
P_10                  	1	0.0000
P_15                  	1	0.0000
P_20                  	1	0.0000
P_30                  	1	0.0000
P_100                 	1	0.0000
P_200                 	1	0.0000
P_500                 	1	0.0000
P_1000                	1	0.0000
num_ret               	10	54
num_rel               	10	1
num_rel_ret           	10	1
map                   	10	0.2500
Rprec                 	10	0.0000


In [37]:
!head -n 1000 $OUTPUT_FILE3

1 Q0 G00-90-0342721 0 22.237840 test
1 Q0 G00-55-3817584 1 13.630871 test
1 Q0 G00-69-2353421 2 7.047234 test
2 Q0 G00-37-1427392 0 19.132774 test
2 Q0 G00-22-3396139 1 18.534701 test
2 Q0 G00-78-1531079 2 15.937579 test
2 Q0 G00-92-0578141 3 15.394800 test
2 Q0 G00-91-1567424 4 15.341260 test
2 Q0 G00-67-0637954 5 15.094297 test
2 Q0 G00-94-1117794 6 13.312850 test
2 Q0 G00-76-0415824 7 12.300545 test
2 Q0 G00-90-3871013 8 12.237758 test
2 Q0 G00-15-1718631 9 10.956456 test
2 Q0 G00-70-2787853 10 9.349964 test
2 Q0 G00-27-2159399 11 9.270222 test
2 Q0 G00-74-1394517 12 3.900235 test
4 Q0 G00-36-1275993 0 15.185037 test
4 Q0 G00-47-2117970 1 14.795664 test
4 Q0 G00-99-2247765 2 14.376807 test
4 Q0 G00-85-1525415 3 13.324419 test
4 Q0 G00-00-1958915 4 13.218041 test
4 Q0 G00-74-4030396 5 12.699734 test
4 Q0 G00-21-2229498 6 12.614776 test
4 Q0 G00-28-2286602 7 12.374009 test
4 Q0 G00-05-1218739 8 12.326272 test
4 Q0 G00-84-3349019 9 12.287600 test
4 Q0 G00-69-0005329 10 12.256160 test
4

In [38]:
!head -n 1000 $QRELS_FILE

1 0 G00-00-0681214 0
1 0 G00-00-0945765 0
1 0 G00-00-1006224 1
1 0 G00-00-1591495 0
1 0 G00-00-2764912 0
1 0 G00-00-3253540 0
1 0 G00-00-3717374 0
1 0 G00-01-0270065 0
1 0 G00-01-0400712 0
1 0 G00-01-0682299 0
1 0 G00-01-2154945 0
1 0 G00-01-2689026 0
1 0 G00-01-2898660 0
1 0 G00-02-0146077 0
1 0 G00-02-0351712 0
1 0 G00-02-0510219 0
1 0 G00-02-0555602 0
1 0 G00-02-0901987 1
1 0 G00-02-1239993 0
1 0 G00-02-3981961 0
1 0 G00-02-4057099 0
1 0 G00-03-0366425 0
1 0 G00-03-0697220 0
1 0 G00-03-0931579 0
1 0 G00-03-1585062 0
1 0 G00-03-1898526 1
1 0 G00-04-0296440 0
1 0 G00-04-0767639 0
1 0 G00-04-0971232 0
1 0 G00-04-1046533 0
1 0 G00-04-1562575 0
1 0 G00-04-1864045 0
1 0 G00-04-3558844 0
1 0 G00-04-4166204 0
1 0 G00-05-0619345 0
1 0 G00-05-0623935 0
1 0 G00-05-1315394 0
1 0 G00-05-2231767 0
1 0 G00-05-2357004 0
1 0 G00-05-2948873 0
1 0 G00-05-2988698 0
1 0 G00-05-3051665 0
1 0 G00-05-3539237 0
1 0 G00-06-0872838 0
1 0 G00-06-2562505 0
1 0 G00-06-3965004 0
1 0 G00-06-4038523 0
1 0 G00-07-11

9 0 G00-09-0395878 0
9 0 G00-09-0826360 0
9 0 G00-09-0888336 0
9 0 G00-09-1310323 0
9 0 G00-09-2693851 0
9 0 G00-09-2834741 0
9 0 G00-09-3229156 0
9 0 G00-09-3470441 0
9 0 G00-09-3580452 0
9 0 G00-09-3931526 0
9 0 G00-10-0136170 0
9 0 G00-10-0818874 0
9 0 G00-10-0994585 0
9 0 G00-10-1232247 0
9 0 G00-10-1843562 0
9 0 G00-10-1931044 0
9 0 G00-10-2011620 0
9 0 G00-10-2753840 0
9 0 G00-10-3392667 0
9 0 G00-10-3849661 0
9 0 G00-10-3892683 0
9 0 G00-11-0770745 0
9 0 G00-12-1926166 0
9 0 G00-13-3039445 0
9 0 G00-14-2961673 0
9 0 G00-16-2086899 0
9 0 G00-16-4067677 0
9 0 G00-17-2521440 0
9 0 G00-17-3027297 0
9 0 G00-18-1660493 0
9 0 G00-18-2665440 0
9 0 G00-19-2902623 0
9 0 G00-20-1794595 0
9 0 G00-20-2822947 0
9 0 G00-20-4189759 0
9 0 G00-21-0118799 0
9 0 G00-21-1529615 0
9 0 G00-21-1973048 0
9 0 G00-21-2737022 0
9 0 G00-22-0400264 0
9 0 G00-22-2107598 0
9 0 G00-23-3963973 0
9 0 G00-24-0016657 0
9 0 G00-25-1916045 0
9 0 G00-26-1048210 0
9 0 G00-28-3598417 0
9 0 G00-30-0221651 0
9 0 G00-31-06

In [39]:
# let count the same words again
myReader2 = INDEX_Q3.reader()
print("# docs with 'juvenile'", myReader2.doc_frequency("file_content", "juvenile"))
print("# docs with 'juvenil'", myReader2.doc_frequency("file_content", "juvenil"))

# docs with 'juvenile' 0
# docs with 'juvenil' 50


### Q3 (c): Provide answer to Q3 

(c) I have used different stems. I got the best result with NLTK LancasterStemmer(). Turning all words to lowercase form, Removing stopwords and punctuations, modifiying each word to its root.  
The overall p5 increased from 0.0714 to 0.1067 with modification. 
Top 5 ranking results for queries 4, 28, 2 , 19 and 10 has been improved by this modification. 

Query 28 that I have mentioned in question 3.a has improved in overall (P10 increased). With NLTK LancasterStemmer, number of FN cases has increased; however, all relevent documents (total relevent doc=2) were retrieved in the top 10 search results => p10=0.2

Also, query 19 that cotains stopword, by removing the stopword and making other modifications, one out of two relevent documents retrieved in the top 5 search results. (improvement)



### Q3 (d): Provide answer to Q3

(d) Yes

### Q3 (e): Provide answer to Q3

(e) Yes, query 22 got worse

### Q3 (f): Provide answer to Q3 

(f) In overall, the modification worked well. Top 5-10 ranking for five query improved, while only one query got worse. Also, P5 has increased significantly for all documents which shows the improvement in ranking. This modification is more efficient than just using vanilla whoosh

## Question 4 (Graduate Students)

In [40]:
GRAD_STUDENT = True # change to True if you are a grad student

### Q4 (a): Provide answer to Q4

(a) I have chosen two FN and FP cases for query 9. There is only one relevent document for this query which is retrieved in the first 20 search results. The total numbers of documents retrieved for this document is 27. We should change the scoring method, so the relevent document stands in top 5 rank.

Since, there was only one document relevent to this query (G00-91-3181951) and it was retrieved, there isn't any false negative case for this query.However, we need to improve the scoring method for that. 

In below, I will provide one example of False positive case. (G00-30-0221651)

The false positive case has high repetition of words in the topic, which makes it as high relevence to the query. The FP case (G00-30-0221651)talks about censuses and compare it with genalogy. Since it uses the word genalogy frequently, the document is consideered as relevent to "9 genalogy searches". 

The goal in this part of assignment is to improve the ranking of True positive cases.

In [41]:
#False Positie case , query 9 , physical therapists

topic_s= r"C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\30\G00-30-0221651"

topic_s= open (topic_s, "r").read().splitlines()
print (topic_s)



['http://www.census.gov/genealogy/www/', '', '   U.S. Census Bureau', '', 'GENEALOGY', '', '  Although censuses are a source of genealogical information, the Census Bureau', '  does not provide these data. The Census Bureau is not able to locate missing', '  persons, or provide recent information on individuals.', '', "   In keeping with the Census Bureau's commitment to confidentiality, the", '   Census Bureau information collected in the Decennial Census of', '   Population and Housing on individuals does not become available to the', '   public until after 72 years.', '', '   If you are looking for information on genealogy please see number 2', '   below and our FAQ.', '', '    1. What the Census Bureau has--', '       The only service we provide related to genealogy is the Age Search', '       Service.', '       In addition, we have two offerings that may be of interest to', '       genealogists. They are counts of names from the 1990 Census. We', '       have no other names data.'

In [42]:
#True Positie case , query 9 , physical therapists

topic_ss= r"C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents\91\G00-91-3181951"

topic_ss= open (topic_ss, "r").read().splitlines()
print (topic_ss)



['http://ferguson.library.ca.gov/html/genealogy.html', '', '', '   State of California Website', '                                      [clearpixel.gif]', '   CSL Catalog', '   Highlights', '   Library of California', '   Only Just Begun', '   Connection - the CSL Newsletter', '   Gold Rush', '   Library Bond Act of 2000', '   Linking Welfare Recipients to Jobs', '   Proposition 218 After Two Years', '   Trustee Toolkit', '   LSTA', '   Civil Liberties Grants', '   Shortcuts', '   Home', '   CRB Reports', '   California Libraries', '   California Family Impact Seminar (CAFIS)', '   Genealogy', '   Library Laws', '   General Info', '   Mission Statement', '   Overview', '   Location/Hours', '   Staff', '   Jobs at the California State Library', '   Search Site', '', '   SELECTED GUIDE TO SOURCES FOR GENEALOGY IN THE CALIFORNIA HISTORY', '   SECTION', '   [clearpixel.gif] [clearpixel.gif] [clearpixel.gif] [clearpixel.gif]', '   [clearpixel.gif] [clearpixel.gif] [clearpixel.gif] [clearpix

In [43]:
print (topics)

['1 mining gold silver coal', '2 juvenile delinquency', '4 wireless communications', '6 physical therapists', '7 cotton industry', '9 genealogy searches', '10 Physical Fitness', '14 Agricultural biotechnology', '16 Emergency and disaster preparedness assistance', '18 Shipwrecks', '19 Cybercrime, internet fraud, and cyber fraud', "22 Veteran's Benefits", '24 Air Bag Safety', '26 Nuclear power plants', '28 Early Childhood Education']


### Q4 (b): Write your code below

In [44]:
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [45]:
Filter4 = RegexTokenizer() | CustomFilter(LancasterStemmer().stem)

mySchema4 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = Filter4))

In [46]:
# if index exists - remove it
if os.path.isdir(INDEX_DIR4):
    shutil.rmtree(INDEX_DIR4)

# create the directory for the index
os.makedirs(INDEX_DIR4)

# create index or open it if already exists
INDEX_Q4  = index.create_in(INDEX_DIR4, mySchema4)

In [47]:
# first we build a list of all the full paths of the files in DOCUMENTS_DIR
filesToIndex = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex.extend(filePaths)

In [48]:
# open writer
myWriter4 = writing.BufferedWriter(INDEX_Q4 , period=None, limit=1000)

try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex):
        with open(filePath, "r", encoding="utf-8") as f:
            fileContent = f.read()
            myWriter4.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter4.close()

already indexed: 1
already indexed: 1001
already indexed: 2001
already indexed: 3001
already indexed: 4001
done indexing.


In [49]:
from whoosh import scoring

In [50]:
#1
#def pos_score_fn(searcher, fieldname, text, matcher):
#    poses = matcher.value_as("positions")
#    return 1.0 / (poses[0] + 1)

# define a query parser for the field "file_content" in the index
#QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema)
#SEARCHER_Q4 = INDEX_Q4.searcher(weighting=scoring.FunctionWeighting(pos_score_fn))
#w = scoring.BM25F(B=0.75, content_B=1.0, K1=1.5)

In [51]:
#2
#QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema)
#SEARCHER_Q4 = INDEX_Q4.searcher(weighting=scoring.BM25F(B=0.3, content_B=1.0, K1=5.2))
#w = scoring.BM25F(B=0.75, content_B=1.0, K1=1.5)

In [52]:
#3
#QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema)
#SEARCHER_Q4 = INDEX_Q4.searcher(weighting=scoring.Frequency())

In [53]:
#4 yesssss,P10 , P15, ... increased with this scoring method
QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema)
SEARCHER_Q4 = INDEX_Q4.searcher(weighting=scoring.PL2())


In [54]:
#5
#QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema)
#SEARCHER_Q4 = INDEX_Q4.searcher(weighting=scoring.TF_IDF())

In [55]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()

# create an output file to which we'll write our results
outputTRECFile4 = open(OUTPUT_FILE3, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
    topicQuery = QP_Q4.parse(topic_phrase)
    topicResults = SEARCHER_Q4.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile4.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile4.close()
topicsFile.close()

In [56]:
print (topics)

['1 mining gold silver coal', '2 juvenile delinquency', '4 wireless communications', '6 physical therapists', '7 cotton industry', '9 genealogy searches', '10 Physical Fitness', '14 Agricultural biotechnology', '16 Emergency and disaster preparedness assistance', '18 Shipwrecks', '19 Cybercrime, internet fraud, and cyber fraud', "22 Veteran's Benefits", '24 Air Bag Safety', '26 Nuclear power plants', '28 Early Childhood Education']


In [81]:
!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE4

num_ret               	1	3
num_rel               	1	5
num_rel_ret           	1	0
map                   	1	0.0000
Rprec                 	1	0.0000
bpref                 	1	0.0000
recip_rank            	1	0.0000
iprec_at_recall_0.00  	1	0.0000
iprec_at_recall_0.10  	1	0.0000
iprec_at_recall_0.20  	1	0.0000
iprec_at_recall_0.30  	1	0.0000
iprec_at_recall_0.40  	1	0.0000
iprec_at_recall_0.50  	1	0.0000
iprec_at_recall_0.60  	1	0.0000
iprec_at_recall_0.70  	1	0.0000
iprec_at_recall_0.80  	1	0.0000
iprec_at_recall_0.90  	1	0.0000
iprec_at_recall_1.00  	1	0.0000
P_5                   	1	0.0000
P_10                  	1	0.0000
P_15                  	1	0.0000
P_20                  	1	0.0000
P_30                  	1	0.0000
P_100                 	1	0.0000
P_200                 	1	0.0000
P_500                 	1	0.0000
P_1000                	1	0.0000
num_ret               	10	42
num_rel               	10	1
num_rel_ret           	10	1
map                   	10	0.2500
Rprec                 	10	0.0000


In [58]:
!head -n 800 $QRELS_FILE

1 0 G00-00-0681214 0
1 0 G00-00-0945765 0
1 0 G00-00-1006224 1
1 0 G00-00-1591495 0
1 0 G00-00-2764912 0
1 0 G00-00-3253540 0
1 0 G00-00-3717374 0
1 0 G00-01-0270065 0
1 0 G00-01-0400712 0
1 0 G00-01-0682299 0
1 0 G00-01-2154945 0
1 0 G00-01-2689026 0
1 0 G00-01-2898660 0
1 0 G00-02-0146077 0
1 0 G00-02-0351712 0
1 0 G00-02-0510219 0
1 0 G00-02-0555602 0
1 0 G00-02-0901987 1
1 0 G00-02-1239993 0
1 0 G00-02-3981961 0
1 0 G00-02-4057099 0
1 0 G00-03-0366425 0
1 0 G00-03-0697220 0
1 0 G00-03-0931579 0
1 0 G00-03-1585062 0
1 0 G00-03-1898526 1
1 0 G00-04-0296440 0
1 0 G00-04-0767639 0
1 0 G00-04-0971232 0
1 0 G00-04-1046533 0
1 0 G00-04-1562575 0
1 0 G00-04-1864045 0
1 0 G00-04-3558844 0
1 0 G00-04-4166204 0
1 0 G00-05-0619345 0
1 0 G00-05-0623935 0
1 0 G00-05-1315394 0
1 0 G00-05-2231767 0
1 0 G00-05-2357004 0
1 0 G00-05-2948873 0
1 0 G00-05-2988698 0
1 0 G00-05-3051665 0
1 0 G00-05-3539237 0
1 0 G00-06-0872838 0
1 0 G00-06-2562505 0
1 0 G00-06-3965004 0
1 0 G00-06-4038523 0
1 0 G00-07-11

In [59]:
print (topics)

['1 mining gold silver coal', '2 juvenile delinquency', '4 wireless communications', '6 physical therapists', '7 cotton industry', '9 genealogy searches', '10 Physical Fitness', '14 Agricultural biotechnology', '16 Emergency and disaster preparedness assistance', '18 Shipwrecks', '19 Cybercrime, internet fraud, and cyber fraud', "22 Veteran's Benefits", '24 Air Bag Safety', '26 Nuclear power plants', '28 Early Childhood Education']


In [60]:
!head -n 60 $OUTPUT_FILE4

1 Q0 G00-90-0342721 0 5.033327 test
1 Q0 G00-55-3817584 1 1.300692 test
1 Q0 G00-69-2353421 2 -0.231048 test
2 Q0 G00-37-1427392 0 5.769230 test
2 Q0 G00-22-3396139 1 5.157296 test
2 Q0 G00-78-1531079 2 4.344075 test
2 Q0 G00-92-0578141 3 3.938257 test
2 Q0 G00-67-0637954 4 3.661064 test
2 Q0 G00-91-1567424 5 3.610061 test
2 Q0 G00-94-1117794 6 3.157254 test
2 Q0 G00-76-0415824 7 2.905243 test
2 Q0 G00-15-1718631 8 2.589093 test
2 Q0 G00-90-3871013 9 2.500874 test
2 Q0 G00-70-2787853 10 2.195349 test
2 Q0 G00-27-2159399 11 1.719913 test
2 Q0 G00-74-1394517 12 0.290776 test
4 Q0 G00-36-1275993 0 4.692110 test
4 Q0 G00-47-2117970 1 4.160913 test
4 Q0 G00-99-2247765 2 3.747074 test
4 Q0 G00-84-3349019 3 3.445406 test
4 Q0 G00-74-4030396 4 3.155766 test
4 Q0 G00-85-1525415 5 3.064534 test
4 Q0 G00-00-1958915 6 3.041380 test
4 Q0 G00-05-1218739 7 2.819956 test
4 Q0 G00-28-2286602 8 2.756261 test
4 Q0 G00-21-2229498 9 2.681662 test
4 Q0 G00-69-0005329 10 2.524930 test
4 Q0 G00-04-3812745 11 

### Q4 (c): Provide answer to Q4

(c)  I have tried 5 different scoring methods for this part of the assignment. PL2 Terrier scoring method increased the overall P5 from 0.1067 to 0.1200. 
P5 for query 9 has improved by changing the scoring method from BM25 to PL2. The relevent document for this query moves from top 20 search results to top 5, which shows improvement.   P5 = 0.2

### Q4 (d): Provide answer to Q4
(d) yes

### Q4 (e): Provide answer to Q4 
(e) Yes, query 28 got worse, while some other queries has improved. Worth mentionening that, the overall p5 improved. 

### Q4 (f): Provide answer to Q4

(f) Yes, since p5 and p10 increased by changing the scoring method, more relevent documents will be retrieved on the first 5-10 search results. This ranking method will be more efficient than the previous one. 

## Validation

In [61]:
# Run the following cells to make sure your code returns the correct value types

In [82]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Path Validation

In [83]:
assert "MATERIALS_DIR" in globals(), "variable MATERIALS_DIR does not exists"
assert(os.path.isdir(os.path.join(MATERIALS_DIR))), "MATERIALS_DIR folder does not exists"
assert(os.path.isdir(os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2"))), "invalid folder structure"
assert(os.path.isdir(os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\documents"))), "invalid folder structure"
print("Paths validated")

Paths validated


### Q2 Validation

In [84]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [85]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation (Graduate Students)

In [86]:
assert((not GRAD_STUDENT) or isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert((not GRAD_STUDENT) or isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert((not GRAD_STUDENT) or isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
